In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import cudf

In [2]:
df = cudf.read_parquet('../../data/train_data/train.parquet')
df['hour'] = df.ts % (60*60*24)
df['day'] = df.ts % (60*60*24*7)
print( df.shape )
df.head()

(163955180, 6)


,session,aid,ts,type,hour,day
0,0,1517085,1659304800,0,79200,338400
1,0,1563459,1659304904,0,79304,338504
2,0,1309446,1659367439,0,55439,401039
3,0,16246,1659367719,0,55719,401319
4,0,1781822,1659367871,0,55871,401471


In [3]:
item_features = df.groupby('aid').agg({'type':'mean','aid':'count','session':'nunique','hour':'mean','day':'mean'})
item_features.columns = ['buy_ratio','count_item','count_user','hour_mean','day_mean']

In [4]:
item_features.head()

,buy_ratio,count_item,count_user,hour_mean,day_mean
aid,,,,,
0,0.000000,33,26,54182.363636,425964.181818
1,0.031250,32,29,50521.250000,266521.250000
2,0.000000,12,12,42171.000000,286971.000000
3,0.079933,1201,668,50329.492090,321759.300583
4,0.035714,140,97,56197.735714,283306.307143


In [5]:
item_features2 = df.groupby('aid').agg({'type':'std','hour':'std','day':'std'}).fillna(-1)
item_features2.columns = ['buy_ratio_std','hour_std','day_std']

f32 = ['buy_ratio_std','hour_std','day_std']
for c in f32: item_features2[c] = item_features2[c].astype('float32')

In [6]:
item_features2.head()

,buy_ratio_std,hour_std,day_std
aid,,,
46071,0.000000,21128.591797,197693.625000
345755,0.249316,19932.507812,155270.156250
1722676,0.000000,15800.455078,75130.781250
1206446,0.353553,14448.579102,191335.015625
178947,0.000000,13722.981445,154929.156250


In [7]:
item_features['repeat'] = item_features.count_item / item_features.count_user

f32 = ['buy_ratio','hour_mean','day_mean','repeat']
for c in f32: item_features[c] = item_features[c].astype('float32')

i32 = ['count_item','count_user']
for c in i32: item_features[c] = item_features[c].astype('int32')

In [8]:
item_features = cudf.concat([item_features,item_features2],axis=1)

In [9]:
item_features.head()

,buy_ratio,count_item,count_user,hour_mean,day_mean,repeat,buy_ratio_std,hour_std,day_std
aid,,,,,,,,,
0,0.000000,33,26,54182.363281,425964.1875,1.269231,0.000000,18119.994141,142572.515625
1,0.031250,32,29,50521.250000,266521.2500,1.103448,0.176777,18789.619141,148777.078125
2,0.000000,12,12,42171.000000,286971.0000,1.000000,0.000000,11595.507812,129372.109375
3,0.079933,1201,668,50329.492188,321759.3125,1.797904,0.314015,16501.742188,167216.656250
4,0.035714,140,97,56197.734375,283306.3125,1.443299,0.186243,15681.410156,174638.421875


In [10]:
item_features.dtypes

buy_ratio        float32
count_item         int32
count_user         int32
hour_mean        float32
day_mean         float32
repeat           float32
buy_ratio_std    float32
hour_std         float32
day_std          float32
dtype: object

In [11]:
item_features.to_parquet('../../data/item_user_features/item4.pqt')